In [2]:
# Libraries

import numpy as np
import pandas as pd
import re
import emoji

import requests
from bs4 import BeautifulSoup
import html
import lxml

# Sacar eventos de secciones

In [22]:
def get_events(tuple_):
    section_, url = tuple_

    # Link to the event
    r = requests.get(url)

    # Pulling the data from the link
    soup = BeautifulSoup(r.text, "lxml")

    # Getting all the events
    events = soup.findAll(class_ = "fv-plan-tile")
    event_images = soup.findAll(class_ = "fv-plan-tile__img")

    # It gets the href attribute for every event in the list
    urls = ["https://feverup.com" + event["href"] for event in events]
    images = [image["src"] for image in event_images]
    sections = [section_ for i in range(len(images))]
    
    return list(zip(sections, urls, images))

In [26]:
section_urls = [("Arte", "https://feverup.com/madrid/cultura-arte-moda"),
                ("Atracciones", "https://feverup.com/madrid/atracciones-tours-viajes"),
                ("Brunch", "https://feverup.com/madrid/brunch"),
                ("Dinner", "https://feverup.com/madrid/find-the-dinner"),
                ("Terrazas", "https://feverup.com/madrid/terrazas-y-rooftops"),
                ("Deportes", "https://feverup.com/madrid/deportes-aventura"),
                ("Conciertos", "https://feverup.com/madrid/conciertos-festivales"),
                ("Actividades", "https://feverup.com/madrid/actividades-juegos"),
                ("Belleza", "https://feverup.com/madrid/actividades-juegos"),
                ("Afterwork", "https://feverup.com/madrid/bebidas-afterwork"),
                ("Clubs", "https://feverup.com/madrid/vida-nocturna-clubs"),
                ("Aire_libre", "https://feverup.com/madrid/planes-aire-libre"),
                ("Candlelights", "https://feverup.com/madrid/candlelight"),
                ("Tributos", "https://feverup.com/madrid/conciertos-tributo"),
                ("Gastronomía", "https://feverup.com/madrid/gastronomia"),
                ("Cine", "https://feverup.com/madrid/cine"),
                ("Teatro", "https://feverup.com/madrid/teatro-comedia-espectaculos")]

arte_urls = get_events(section_urls[0])

# Procesador de eventos

In [3]:
#### Function 1
def get_data(url):
    # Link to the event
    r = requests.get(url)
    # Pulling the data from the link
    soup = BeautifulSoup(r.text, "lxml")
    # Take event title and description
    # In the event description is where "accesibility" info is located
    event_title = soup.find(class_ = "plan-hero__title").text
    event_descr = soup.find(class_ = "plan-description mb-32")
    # Transform the event description into string for later processing
    event_descr = str(event_descr)
    # Get price info
    event_price = soup.find(class_ = "sidebarBuyingText sidebarWrapper__btn").text
    event_price = event_price.split("\xa0€")[0]

    return event_title, event_descr, event_price

#### Function 2
def separate_sections(event_descr):
    # 1) Split the data using the "<strong>" tag -> This way we separate the sections
    # 2) Split the data using the "</strong>" tag -> This way we separate titles from descriptions
    event_descr_items = [elem.split("</strong>") for elem in event_descr.split("<strong>")]

    return event_descr_items

#### Function 3
def separate_title_descr(event_descr_items):
    # Dict to save the info title-descriptions that we have in a list of lists
    title_descr = {}
    # This is to assign a numerical value as key to those descriptions without section title
    no_title_count = 1

    # Iterate over the list of lists, and for every list...
    for elem in event_descr_items:
        # If there's more than one element (that means, we have description and title)...
        if len(elem) > 1:
            # Then the first element will be the key and the second one will be the value in our new dict
            title_descr[elem[0]] = elem[1]
        # If we don't have two values (we are missing the title)...
        else:
            # Then, the key will be the numerical value we defined and the value will be the only value of the list, which should be the description
            title_descr[no_title_count] = elem[0]
            no_title_count += 1

    return title_descr

#### Function 4
def remove_html(title_descr):
    # This is to remove all the html tags from the text
    for key, value in title_descr.items():
        title_descr[key] = re.sub(r"\<.*?\>", "", value)

    return title_descr
    
#### Function 5
def remove_emojis(string, emoji_list):
    # We'll save all the actual characters (not emojis) here
    new_string = []
    # Iterate over the string
    for s in string:
        # If character not in the emoji list
        if s not in emoji_list:
            # append it to the new_string list
            new_string.append(s)
    
    # join the list into a new string
    return "".join(map(str, new_string))

#### Function 6
def remove_all_emojis(title_descr_without_html, emojis, info_names):
    
    for key, val in title_descr_without_html.items():
        if key in info_names:
            pass
        else:
            title_descr_without_html[key] = remove_emojis(title_descr_without_html[key], emojis)

    return title_descr_without_html

#### Function 7
def separate_info_sections(title_descr_without_html, info_names, icons_list):
    # Now I have to do all the processing with the iterators to pull the information from "Información general".

    # We'll need an iterator
    # iterator = new_dict["Información General"]
    for name in info_names:
        try:
            iterator = title_descr_without_html[name]
            break
        except:
            pass

    # We'll need an empty list too, save the subsections
    general_info_sections = []

    # Iterate over all the icons in the list
    for icon in icons_list:
        # If the icon is in the iterator (string)...
        if icon in iterator:
            # Split iterator using the icon and save it as the new iterator
            # Input: string
            # Output: list with 2 items
            iterator = iterator.split(icon)
            # Save the first item in our new list
            general_info_sections.append(iterator[0])

            # Check if there is still any icon left in the second element of the iterator
            if any(icon in iterator[1] for icon in icons_list):
                # If so, save it as the new iterator (a string again)
                iterator = iterator[1]

            # Else, save it in our new list.
            # As we don't have more icons in the second element, that means, we don't need to split the iterator anymore, since we already reach the last piece of info we needed
            else:
                general_info_sections.append(iterator[1])
        else:
            pass

    return general_info_sections

#### Function 8
def separate_info_sections_title_descr(general_info_sections):
    
    new_general_info_sections = []

    for elem in general_info_sections:
        if len(elem) > 1:
            new_general_info_sections.append(elem.split(":", 1))

    return new_general_info_sections

#### Function 9
def transform_info_sections(new_general_info_sections):
    sections = {}
    extra = 1

    for list_ in new_general_info_sections:
        if len(list_) > 1:
            sections[list_[0]] = list_[1]
        else:
            sections[extra] = list_[0]
            extra += 1

    return sections

#### Function 10
def add_rest(sections, event_title, event_price):
    rest = {"Event_title": event_title, "Event_price": event_price}
    final_dict = {**sections, **rest}
    return final_dict

#### Function 11
def create_df(dict1, dict2, index):
    # Creating dfs
    main_sections_df = pd.DataFrame(dict1, index = [index])
    subsections_df = pd.DataFrame(dict2, index = [index])

    # Joining dfs
    full_df = pd.merge(main_sections_df, subsections_df, how = "outer", left_index = True, right_index = True)

    return full_df

In [39]:
def processor(source, emojis, section_emojis, info_names, index):
    section, url, image = source

    # Step 1: Get data
    try:
        event_title, event_descr, event_price = get_data(url)
    except:
        return "Step 1 error"

    # Step 2: Split event_descr into sections
    try:
        event_descr_items = separate_sections(event_descr)
    except:
        return "Step 2 error"

    # Step 3: Split sections into titles and descriptions
    try:
        title_descr = separate_title_descr(event_descr_items)
    except:
        return "Step 3 error"

    # Step 4: Some cleaning
    # 4.1 Remove html
    try:
        title_descr_without_html = remove_html(title_descr)
    except:
        return "Step 4.1 error"
    # 4.2 Remove emojis
    try:
        cleaned_dict = remove_all_emojis(title_descr_without_html, emojis, info_names)
    except:
        return "Step 4.2 error"

    # Step 5: Get data from "info" section
    try:
        general_info_sections = separate_info_sections(title_descr_without_html, info_names, section_emojis)
    except:
        return "Step 5 error"
    # 5.1 Split the data from "info" section into title and descr
    try:
        new_general_info_sections = separate_info_sections_title_descr(general_info_sections)
    except:
        return "Step 5.1 error"

    # Step 6: Make the "info" subsections a list
    try:
        info_sections = transform_info_sections(new_general_info_sections)
    except:
        return "Step 6 error"

    # Step 7: Join all the data together into a dict
    try:
        final_dict = add_rest(info_sections, event_title, event_price)
    except:
        return "Step 7 error"

    # Step 8: Create joined dataframe
    try:
        df = create_df(cleaned_dict, final_dict, index)
    except:
        return "Step 8 error"

    # Step 9: Add the event url and image
    try:
        df["url"] = url
        df["image"] = image
        df["event_type"] = section
    except:
        return "Step 9 error"

    return df

url = "https://feverup.com/m/98148"
emojis = emoji.UNICODE_EMOJI["en"]
section_emojis = ["📅", "🕒", "⏳", "👤", "📍", "⚠️", "♿", "⌚", "❓", "🔗"]
info_names = ["Información", "Información General"]
index = 0

processor(url, emojis, section_emojis, info_names, index)

In [31]:
def multiple_processor(sources, emojis, section_emojis, info_names, index):
    dfs = []

    for source_ in sources:
        df_ = processor(source_, emojis, section_emojis, info_names, index)

        if type(df_) != str:
            dfs.append(df_)
            print(index, source_)
        else:
            print(index, source_, df_)
        index += 1
        
    return pd.concat(dfs)

In [40]:
emojis = emoji.UNICODE_EMOJI["en"]
section_emojis = ["📅", "🕒", "⏳", "👤", "📍", "⚠️", "♿", "⌚", "❓", "🔗"]
info_names = ["Información", "Información General", "Información:", "Información general"]
index = 0

dfs = []

for section in section_urls:
    urls = get_events(section)
    df = multiple_processor(urls, emojis, section_emojis, info_names, index)
    dfs.append(df)
    print("-" * 20, "Section done", "-" * 20)

0 ('Arte', 'https://feverup.com/m/100122', 'https://fever.imgix.net/plan/photo/0831cb1f-cf6d-11eb-9522-06551cb39bc6.jpg?auto=compress&auto=format&fm=jpg&w=720&h=720')
1 ('Arte', 'https://feverup.com/m/97759', 'https://fever.imgix.net/plan/photo/8fcd59b1-c3ab-11eb-9522-06551cb39bc6.jpg?auto=compress&auto=format&fm=jpg&w=720&h=720')
2 ('Arte', 'https://feverup.com/m/96379', 'https://fever.imgix.net/plan/photo/15de4ab8-7d94-11eb-b783-06551cb39bc6.jpg?auto=compress&auto=format&fm=jpg&w=720&h=720')
3 ('Arte', 'https://feverup.com/m/87806', 'https://fever.imgix.net/plan/photo/71da0628-b6ec-11ea-bf03-06551cb39bc6.jpg?auto=compress&auto=format&fm=jpg&w=720&h=720')
4 ('Arte', 'https://feverup.com/m/99841', 'https://fever.imgix.net/plan/photo/5e0576af-cd00-11eb-9522-06551cb39bc6.jpg?auto=compress&auto=format&fm=jpg&w=720&h=720')
5 ('Arte', 'https://feverup.com/m/98148', 'https://fever.imgix.net/plan/photo/2ede03b8-adc1-11eb-9520-06551cb39bc6.jpg?auto=compress&auto=format&fm=jpg&w=720&h=720')
6 (

In [44]:
count = 1
for df in dfs:
    df.to_excel(f"dfs/df_{count}.xlsx")
    count += 1

In [59]:
dfs[12].loc[:, "event_type"]

0    Candlelights
1    Candlelights
2    Candlelights
3    Candlelights
4    Candlelights
5    Candlelights
6    Candlelights
7    Candlelights
8    Candlelights
9    Candlelights
Name: event_type, dtype: object

In [60]:
pd.read_excel("dfs/df_1.xlsx")

,Unnamed: 0,1_x,Tickets,Qué vas a disfrutar,Información General,Descripción,Fechas,Horarios,Duración,Edad,...,Fecha y hora,Información importante,Fecha,Ficha técnica,📌 Lugar,Agatha Ruiz de la Prada u Okuda San Miguel,Edad mínima,Duración aproximada del recorrido,Edad mínima recomendada,El musical del Rey León tiene una duración de 2
0,0,Meet Vincent van Gogh es una experiencia trid...,Entrada - incluye acceso a la exhibición y au...,Descubre una nueva forma de admirar las obras...,📅 Fechas: a partir de septiembre de 2021🕒 Hora...,¿Alguna vez has soñado con adentrarte en una p...,a partir de septiembre de 2021,-Lunes a jueves y domingos: de 10:00 a 20:00-V...,60 minutos aproximadamente,¡todas las edades! (menores de 14 años acompa...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,Entrada para 1 adulto (a partir de 12 años) E...,Un museo inmersivo con proyecciones y arte i...,NaN,Velázquez Tech Museum es una experiencia senso...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,Entrada general para la exposición Superthing...,Más de 1.000 m2 y diferentes escenarios Fotos...,NaN,"¡Por primera vez a nivel internacional, sumérg...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,Entrada de adulto (a partir de 13 años) Entra...,NaN,NaN,¡Ven y conoce el misterioso mundo de las ilusi...,NaN,NaN,NaN,NaN,...,escoge la opción que quieras al comprar,el museo cuenta con acceso para personas con ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,"Entrada a la exposición Dino World, un fantás...",Un viaje didáctico al periodo jurásico con im...,NaN,¡Llega a IFEMA Madrid la gran exposición inter...,NaN,NaN,una hora aproximadamente,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,NaN,"Apúntate a ""Concierto de Pinturilla y La Pand...",NaN,NaN,Pinturilla y la Pandilla Vainilla es un nuevo ...,NaN,NaN,NaN,todos los públicos💰 Precio estimado de 10€ a 22€,...,NaN,NaN,domingos de junio por la mañana📍 Lugar: Espac...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,NaN,Entrada Adulto (de 13 a 65 años) para 1 día e...,"El acuario más grande de Europa, dentro de la...",NaN,¡Nunca has visto nada igual! El Oceanogràfic e...,NaN,NaN,NaN,NaN,...,escoge la opción que quieras al comprar📍 Luga...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,NaN,Entrada general para 1 persona para ¿Nos casa...,NaN,NaN,¿Nos casamos? es un pieza teatral enmarcada en...,NaN,NaN,aproximadamente 60 minutos,NaN,...,NaN,NaN,NaN,Actores: Daniel Molina y Paola MarquinezAutor...,Soho Club Teatro🕐 Fecha y hora: escoge la opc...,NaN,NaN,NaN,NaN,NaN
8,8,NaN,Entrada general (a partir de 12 años) Entrada...,Acceso a un viaje a la imaginación que une el...,NaN,Disfruta de una aventura interactiva en Sweet ...,escoge el día que quieras al comprar,escoge la opción que quieras al comprar📍 Luga...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,se han unido para exponer piezas e historias ...,sin restricción,45 minutos/ 1 hora,NaN,NaN
9,9,NaN,Tarjeta regalo por valor de 100€ Tarjeta rega...,NaN,NaN,¡Les dejarás sin palabras! Si quieres sorprend...,podrás canjear tu tarjeta desde el momento de...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6 años📱 Encontrarás información detallada sob...,45 horas incluyendo un intermedio de 20 minutos


# Candlelights

In [18]:
#candlelight_urls = ['https://feverup.com/m/96831', 'https://feverup.com/m/97975', 'https://feverup.com/m/97053', 'https://feverup.com/m/100404', 'https://feverup.com/m/100896', 'https://feverup.com/m/95040', 'https://feverup.com/m/100405', 'https://feverup.com/m/98160', 'https://feverup.com/m/97280', 'https://feverup.com/m/84521']
candlelight = [('https://feverup.com/m/84521',
  'https://fever.imgix.net/plan/photo/629c34a6-c217-11eb-9522-06551cb39bc6.jpg?auto=compress&auto=format&fm=jpg&w=720&h=720'),
 ('https://feverup.com/m/96831',
  'https://fever.imgix.net/plan/photo/1211c7ea-87fd-11eb-b783-06551cb39bc6.jpg?auto=compress&auto=format&fm=jpg&w=720&h=720'),
 ('https://feverup.com/m/97053',
  'https://fever.imgix.net/plan/photo/0aa036e0-8e1d-11eb-b783-06551cb39bc6.jpg?auto=compress&auto=format&fm=jpg&w=720&h=720'),
 ('https://feverup.com/m/100404',
  'https://fever.imgix.net/plan/photo/ffbef88e-d434-11eb-9522-06551cb39bc6.jpg?auto=compress&auto=format&fm=jpg&w=720&h=720'),
 ('https://feverup.com/m/100896',
  'https://fever.imgix.net/plan/photo/6ef77030-df18-11eb-b9f7-6a68b27be3f2.jpg?auto=compress&auto=format&fm=jpg&w=720&h=720'),
 ('https://feverup.com/m/100405',
  'https://fever.imgix.net/plan/photo/aa66c2cf-d437-11eb-9522-06551cb39bc6.jpg?auto=compress&auto=format&fm=jpg&w=720&h=720'),
 ('https://feverup.com/m/97280',
  'https://fever.imgix.net/plan/photo/32e05ec6-96bc-11eb-b783-06551cb39bc6.jpg?auto=compress&auto=format&fm=jpg&w=720&h=720'),
 ('https://feverup.com/m/98160',
  'https://fever.imgix.net/plan/photo/8e120dda-adaf-11eb-9520-06551cb39bc6.jpg?auto=compress&auto=format&fm=jpg&w=720&h=720'),
 ('https://feverup.com/m/100980',
  'https://fever.imgix.net/plan/photo/c9403492-e0c0-11eb-a5a2-f29321d0df9f.jpg?auto=compress&auto=format&fm=jpg&w=720&h=720'),
 ('https://feverup.com/m/97975',
  'https://fever.imgix.net/plan/photo/8268fa5a-a73a-11eb-9520-06551cb39bc6.jpg?auto=compress&auto=format&fm=jpg&w=720&h=720')]

emojis = emoji.UNICODE_EMOJI["en"]
section_emojis = ["📅", "🕒", "⏳", "👤", "📍", "⚠️", "♿", "⌚", "❓", "🔗"]
info_names = ["Información", "Información General", "Información:", "Información general"]
index = 0

df_1 = multiple_processor(candlelight, emojis, section_emojis, info_names, index)

0 ('https://feverup.com/m/84521', 'https://fever.imgix.net/plan/photo/629c34a6-c217-11eb-9522-06551cb39bc6.jpg?auto=compress&auto=format&fm=jpg&w=720&h=720')
1 ('https://feverup.com/m/96831', 'https://fever.imgix.net/plan/photo/1211c7ea-87fd-11eb-b783-06551cb39bc6.jpg?auto=compress&auto=format&fm=jpg&w=720&h=720')
2 ('https://feverup.com/m/97053', 'https://fever.imgix.net/plan/photo/0aa036e0-8e1d-11eb-b783-06551cb39bc6.jpg?auto=compress&auto=format&fm=jpg&w=720&h=720')
3 ('https://feverup.com/m/100404', 'https://fever.imgix.net/plan/photo/ffbef88e-d434-11eb-9522-06551cb39bc6.jpg?auto=compress&auto=format&fm=jpg&w=720&h=720')
4 ('https://feverup.com/m/100896', 'https://fever.imgix.net/plan/photo/6ef77030-df18-11eb-b9f7-6a68b27be3f2.jpg?auto=compress&auto=format&fm=jpg&w=720&h=720')
5 ('https://feverup.com/m/100405', 'https://fever.imgix.net/plan/photo/aa66c2cf-d437-11eb-9522-06551cb39bc6.jpg?auto=compress&auto=format&fm=jpg&w=720&h=720')
6 ('https://feverup.com/m/97280', 'https://fever.

In [19]:
# 1) Generate full candlelights dataframe
candlelights_df = df_1

# 2) Let's start filling NaNs and combining columns
# NaN filling
candlelights_df.loc[:, ["Intérprete:", "Intérpretes:"]] = candlelights_df.loc[:, ["Intérprete:", "Intérpretes:"]].fillna("")
candlelights_df.loc[:, ["Información:", "Información general"]] = candlelights_df.loc[:, ["Información:", "Información general"]].fillna("")
candlelights_df.loc[:, [" Duración aproximada", " Duración del concierto", " Duración"]] = candlelights_df.loc[:, [" Duración aproximada", " Duración del concierto", " Duración"]].fillna("")
candlelights_df.loc[:, ["Valoraciones de otros usuarios", "Valoraciones de usuarios de otros conciertos Candlelight"]] = candlelights_df.loc[:, ["Valoraciones de otros usuarios", "Valoraciones de usuarios de otros conciertos Candlelight"]].fillna("")

# Columns combination
candlelights_df["Interprete"] = candlelights_df["Intérprete:"].astype(str) + candlelights_df["Intérpretes:"].astype(str)
#candlelights_df["Informacion"] = candlelights_df["Información:"].astype(str) + candlelights_df["Información general"].astype(str)
candlelights_df["Duracion"] = candlelights_df[" Duración aproximada"].astype(str) + candlelights_df[" Duración del concierto"].astype(str) + candlelights_df[" Duración"].astype(str)
candlelights_df["Valoraciones"] = candlelights_df["Valoraciones de otros usuarios"].astype(str) + candlelights_df["Valoraciones de usuarios de otros conciertos Candlelight"].astype(str)

# 3) Drop columns
candlelights_df = candlelights_df.drop(["Intérprete:", "Intérpretes:", "Información:", "Información general", "El uso de la mascarilla es obligatorio durante el conciertoPrograma", " La seguridad de nuestros asistentes es una prioridad para Fever. Puedes consultar las medidas de seguridad de este evento aquíPrograma", "El uso de la mascarilla es obligatorio durante el conciertoPrograma", 1, "1_x", "1_y", 2, " Duración aproximada", " Duración del concierto", " Duración", "Valoraciones de otros usuarios", "Valoraciones de usuarios de otros conciertos Candlelight"], axis = 1)
candlelights_df = candlelights_df.rename({"📍 Lugar" : "Lugar"}, axis = 1)

print(candlelights_df.shape)
candlelights_df

(10, 15)


,Tickets,Qué vas a disfrutar,Descripción,Lugar,Fecha y hora,Edad,Accesibilidad,Event_title,Event_price,url,image,Programa,Interprete,Duracion,Valoraciones
0,Entrada zona A: mejor visibilidad Entrada zon...,️ Una atmósfera muy íntima en una espectacular...,¡Disfruta de las mejores bandas sonoras del ci...,Teatro Goya,varias opciones a las 21:30,a partir de 8 años. Los menores de 16 años de...,recinto habilitado para personas en silla de ...,Candlelight Open Air: Morricone y otras banda...,"25,00",https://feverup.com/m/84521,https://fever.imgix.net/plan/photo/629c34a6-c2...,NaN,Cuarteto de cuerda - Matrice,65 minutos aprox. (apertura de puertas 60 min...,"Clarines V. : ""La elección de bandas sonoras ..."
1,Entrada zona A: muy buena visibilidad Entrada...,️ Una atmósfera muy íntima en una localización...,"¡Las mejores canciones de tu infancia, como nu...",Teatro Goya,7 de agosto a las 21:30,a partir de 8 años. Los menores de 16 años de...,recinto habilitado para personas en silla de ...,Candlelight Open Air: Bandas Sonoras Mágicas,"15,00",https://feverup.com/m/96831,https://fever.imgix.net/plan/photo/1211c7ea-87...,NaN,,60 minutos (apertura de puertas 60 minutos an...,"Dre S. : ""Muy buena idea y excelente conciert..."
2,Entrada zona A: mejor visibilidad Entrada zon...,️ Una atmósfera muy íntima en una espectacular...,"¡Los mejores temas de Daft Punk, bajo la luz d...","El Capricho, Teatro Goya",28 de agosto a las 21:30,a partir de 8 años. Los menores de 16 años de...,recinto habilitado para personas en silla de ...,Candlelight Open Air: Tributo a Daft Punk a l...,"25,00",https://feverup.com/m/97053,https://fever.imgix.net/plan/photo/0aa036e0-8e...,NaN,Piano y violín eléctrico - Laura Andrés y Rose...,60 minutos aprox. (apertura de puertas 60 min...,"Clara S. : ""Todo fantástico. Organización y m..."
3,Entrada zona A: muy buena visibilidad Entrada...,️ Una atmósfera muy íntima en una espectacular...,¡Disfruta de música clásica como nunca antes e...,"Círculo de Bellas Artes, Sala Columnas","9, 11, 17 o 19 de septiembre a las 19:00 o la...",a partir de 8 años. Los menores de 16 años de...,recinto habilitado para personas en silla de ...,Candlelight: El Cascanueces y ballet a la luz...,"15,00",https://feverup.com/m/100404,https://fever.imgix.net/plan/photo/ffbef88e-d4...,"El Lago de los Cisnes, TchaikovskySuite del Ca...",Cuarteto de cuerda - Matrice,70 minutos aprox. (apertura de puertas 45 min...,"Raquel H. : ""Precioso, romántico e íntimo. Un..."
4,Entrada zona A: mejor visibilidad Entrada zon...,️ Una atmósfera muy íntima en una espectacular...,¡Revive a las leyendas del jazz a la luz de la...,Teatro Goya,6 de agosto a las 21:30,a partir de 8 años. Los menores de 16 años de...,recinto habilitado para personas en silla de ...,Candlelight Jazz Open Air: Tributo a Nina Sim...,"25,00",https://feverup.com/m/100896,https://fever.imgix.net/plan/photo/6ef77030-df...,NaN,Contrabajo: Sergio FernándezBatería: Martin Da...,60 minutos aprox. (apertura de puertas 60 min...,"Sofía A. : ""Una atmósfera muy acogedora y mús..."
5,Entrada zona A: mejor visibilidad Entrada zon...,️ Una atmósfera muy íntima en una espectacular...,¡Disfruta de las mejores bandas sonoras del ci...,Círculo de Bellas Artes - Sala Columnas,10 de septiembre a las 19:00 y las 21:00,a partir de 8 años. Los menores de 16 años de...,recinto habilitado para personas en silla de ...,"Candlelight: Bandas Sonoras, cuarteto de cuer...","15,00",https://feverup.com/m/100405,https://fever.imgix.net/plan/photo/aa66c2cf-d4...,Indiana Jones de J. WilliamsCinema Paradiso de...,Cuarteto de cuerda - Matrice,60 minutos aprox. (apertura de puertas 45 min...,"Sofía A. : ""Una atmósfera muy acogedora, y un..."
6,Entrada zona A: muy buena visibilidad Entrada...,️ Una atmósfera muy íntima en una espectacular...,Siente la fuerza y emoción de las voces lírica...,Círculo de Bellas Artes - Sala Columnas,18 de septiembre a las 19:00 y las 21:00,a partir de 8 años. Los menores de 16 años de...,recint

In [20]:
candlelights_df.to_excel("Candlelights.xlsx")

# Art events

In [63]:
art_urls = ['https://feverup.com/m/97759', 'https://feverup.com/m/87806', 'https://feverup.com/m/92796',
             'https://feverup.com/m/92640', 'https://feverup.com/m/96367', 'https://feverup.com/m/94382',
             'https://feverup.com/m/92141']

emojis = emoji.UNICODE_EMOJI["en"]
section_emojis = ["📅", "🕒", "⏳", "👤", "📍", "⚠️", "♿", "⌚", "❓", "🔗"]
info_names = ["Información", "Información General", "Información:", "Información general"]
index = 0

df_2 = multiple_processor(art_urls, emojis, section_emojis, info_names, index)
print(df_2.shape)

0 https://feverup.com/m/97759
1 https://feverup.com/m/87806
2 https://feverup.com/m/92796
3 https://feverup.com/m/92640
4 https://feverup.com/m/96367
5 https://feverup.com/m/94382
6 https://feverup.com/m/92141
(7, 28)


In [80]:
art_df = df_2

# 2) Let's start filling NaNs and combining columns
# NaN filling
art_df.loc[:, ["📍 Lugar", "📌 Lugar", " Lugar"]] = art_df.loc[:, ["📍 Lugar", "📌 Lugar", " Lugar"]].fillna("")
art_df.loc[:, ["Qué vas a disfrutar", "Cosas alucinantes que disfrutarás"]] = art_df.loc[:, ["Qué vas a disfrutar", "Cosas alucinantes que disfrutarás"]].fillna("")
art_df.loc[:, ["Valoraciones de usuarios", "Valoraciones", "Valoraciones sobre el trabajo de Pablo Raijenstein"]] = art_df.loc[:, ["Valoraciones de usuarios", "Valoraciones", "Valoraciones sobre el trabajo de Pablo Raijenstein"]].fillna("")
art_df.loc[:, [" Edad mínima", " Edad"]] = art_df.loc[:, [" Edad mínima", " Edad"]].fillna("")
art_df.loc[:, [" Duración aproximada del recorrido", " Duración"]] = art_df.loc[:, [" Duración aproximada del recorrido", " Duración"]].fillna("")

# Columns combination
art_df["Lugar"] = art_df["📍 Lugar"].astype(str) + art_df["📌 Lugar"].astype(str) + art_df[" Lugar"].astype(str)
art_df["Que vas a disfrutar"] = art_df["Qué vas a disfrutar"].astype(str) + art_df["Qué vas a disfrutar"].astype(str) + art_df["Cosas alucinantes que disfrutarás"].astype(str)
art_df["Valoraciones"] = art_df["Valoraciones de usuarios"].astype(str) + art_df["Valoraciones"].astype(str) + art_df["Valoraciones sobre el trabajo de Pablo Raijenstein"].astype(str)
art_df["Edad"] = art_df[" Edad mínima"].astype(str) + art_df[" Edad"].astype(str)
art_df["Duracion"] = art_df[" Duración aproximada del recorrido"].astype(str) + art_df[" Duración"].astype(str)

# 3) Drop columns
art_df = art_df.drop([1, "📍 Lugar", "📌 Lugar", " Lugar", "Qué vas a disfrutar",
                     "Cosas alucinantes que disfrutarás", "Valoraciones de usuarios",
                     "Valoraciones sobre el trabajo de Pablo Raijenstein", "1_x", "1_y",
                     "🕐 Fecha y hora", " Fechas", " Horarios", "Agatha Ruiz de la Prada u Okuda San Miguel",
                     " Edad mínima", " Edad", " Duración aproximada del recorrido", " Duración",
                     "Información", "Información general"], axis = 1)

art_df = art_df.fillna("")
art_df = art_df[art_df.loc[:, " Accesibilidad"] != ""]

art_df = art_df.drop(["Valoraciones", "Que vas a disfrutar", "Edad", "Duracion"], axis = 1)
print(art_df.shape)
art_df

(1, 8)


,Tickets,Descripción,Event_title,Event_price,Fecha y hora,Información importante,Accesibilidad,Lugar
5,Entrada adulto (de 17 a 60 años) Entrada seni...,El Museo Lázaro Galdiano es el resultado del l...,Entradas para el Museo Lázaro Galdiano,"4,50",,existencia de medidas de seguridad debidas al...,accesible con silla de ruedas,Museo Lázaro Galdiano⌛ Duración aproximada de...


In [81]:
art_df.to_excel("Art_events.xlsx")

# Outdoor plans

In [83]:
outdoor_urls = ['https://feverup.com/m/100044', 'https://feverup.com/m/100978', 
                'https://feverup.com/m/100603', 'https://feverup.com/m/100497',
                'https://feverup.com/m/99411', 'https://feverup.com/m/100896',
                'https://feverup.com/m/74440', 'https://feverup.com/m/98057',
                'https://feverup.com/m/93402', 'https://feverup.com/m/80284']

emojis = emoji.UNICODE_EMOJI["en"]
section_emojis = ["📅", "🕒", "⏳", "👤", "📍", "⚠️", "♿", "⌚", "❓", "🔗"]
info_names = ["Información", "Información General", "Información:", "Información general"]
index = 0

df_3 = multiple_processor(outdoor_urls, emojis, section_emojis, info_names, index)
print(df_3.shape)

0 https://feverup.com/m/100044
1 https://feverup.com/m/100978
2 https://feverup.com/m/100603
3 https://feverup.com/m/100497
4 https://feverup.com/m/99411
5 https://feverup.com/m/100896
6 https://feverup.com/m/74440
7 https://feverup.com/m/98057
8 https://feverup.com/m/93402
9 https://feverup.com/m/80284
(10, 31)


In [84]:
df_3

,1_x,Tickets,Qué vas a disfrutar,Información,Descripción,Valoraciones de otros usuarios,Fecha,Horarios,Lugar,1_y,...,Fecha y hora,Duración,1,Información general,Edad mínima,Información de envío,Kit Bisolé,Kit Menorquín:,🕐 Fecha de envío,Se deberá abonar una fianza que será devuelta a la entrega del vehículo tras la experienciaRutas
0,,Entrada Adulto para 1 día completo o media jo...,Un refrescante chapuzón en la enorme piscina ...,📅 Fecha: todos los días🕒 Horarios: Entrada dí...,¡Comienza la temporada de piscina Club de Tiro...,"Marta H. : Piscina limpia, césped cuidado, pe...",todos los días,Entrada día completo: de 11:00 a 20:30 Entra...,Club de Tiro de El Pardo🚗 Parking exclusivo p...,No está permitido introducir comida o bebida a...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,Entrada general + 2 consumiciones hasta las 2...,Una fiesta en la Terraza de la discoteca más ...,📅 Fecha: sábado 7 de agosto🕒 Horario: de 19:30...,¡Vete preparando porque el sábado 7 de agosto...,NaN,sábado 7 de agosto,NaN,NaN,Es obligatorio el uso de mascarilla en todo m...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,,Entrada general + 2 consumiciones hasta las 2...,Una fiesta Remember increíble en la discoteca...,📅 Fecha: sábado 24 de julio🕒 Horario: de 19:30...,"Esperamos que estés descansando bien, porque e...",NaN,sábado 24 de julio,NaN,NaN,Es obligatorio el uso de mascarilla en todo m...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,,Entrada general + 2 consumiciones hasta las 2...,La vuelta de la fiesta hardstyle de referenci...,📅 Fecha: sábado 31 de julio🕒 Horario: de las 1...,Una de tus fiestas de referencia aterriza de n...,NaN,sábado 31 de julio,NaN,NaN,Es obligatorio el uso de mascarilla en todo m...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,,"Acceso a Corona Paradise, una experiencia sen...",Coge tu mochila y prepara tu cámara porque… ¡...,📅 Fechas: del 1 de julio al 1 de agosto🕒 Horar...,Llega a Madrid la experiencia más divertida y...,NaN,NaN,varias opciones disponibles📍 Lugar: Palacio d...,NaN,La seguridad de nuestros asistentes es una p...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,,Entrada zona A: mejor visibilidad Entrada zon...,️ Una atmósfera muy íntima en una espectacular...,NaN,¡Revive a las leyendas del jazz a la luz de la...,"Sofía A. : ""Una atmósfera muy acogedora y mús...",NaN,NaN,NaN,El uso de la mascarilla es obligatorio durante...,...,6 de agosto a las 21:30,60 minutos aprox. (apertura de puertas 60 min...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,"Ruta a caballo para 1 persona de 60, 90 o 120...",NaN,NaN,¡Siente el aire puro de la sierra a lomos de u...,NaN,NaN,NaN,NaN,NaN,...,sábados y domingos excepto del 1 al 25 de ago...,NaN,,🕒 Fecha y hora: sábados y domingos excepto del...,mayores de 4 años,NaN,NaN,NaN,NaN,NaN
7,NaN,Kit Bisolé + tabla de madera + bolsa de tela ...,NaN,🕐 Fecha de envío:Los envíos solicitados antes ...,¡Disfruta de los mejores aperitivos gourmet en...,NaN,NaN,NaN,tu casa📱 Encontrarás más detalles sobre tu re...,NaN,...,NaN,NaN,,NaN,NaN,La distribución está disponible en todo el te...,Aperitivos:Un bote de aceitunas partidas verde...,Aperitivos:Una bolsa de patatas fritas (140 g)...,Los envíos solicitados antes de las 17:00 de l...,NaN
8,NaN,Ruta guiada Madrid Encantado para 1 adulto (a...,NaN,🕒 Fecha y hora: escoge la opción que prefieras...,Conocer Madrid durante la noche es una de las...,NaN,NaN,NaN,NaN,NaN,...,escoge la opción que prefieras al comprar⌚ Du...,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,Night Tour de 2 horas por Madrid conduciendo ...,NaN,🕒 Fecha: escoge la opción que quieras al compr...,Descubre los lugares más míticos de Madrid des...,NaN,escoge la opción que quieras al comprar📍 Luga...,NaN,NaN,NaN,...,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,"Night Tour: el Madrid más castizo, bajo la luz..."
